# Extract Bin Files

This notebook will check for the previously extracted FOVs in an image directory and proceed to extract any remaining FOVs from the provided base directory.

In [ ]:
import os

from toffy.panel_utils import load_panel, modify_panel_ranges
from toffy.bin_extraction import extract_missing_fovs, incomplete_fov_check

## Required variables
You will need to define the following two arguments for this notebook.
 - `run_name` should contain the exact name of the MIBI run to extract from
 - `panel_path` should point to a panel csv specifying the targets on your panel. You can download your panel online from the Ionpath MibiTracker and then copy the file to the `C:\\Users\\Customer.ION\\Documents\panel_files` directory (see [panel format](https://github.com/angelolab/toffy#panel-format) for more information)
 - `extract_prof` indicates whether you want to include mass proficient extraction on top of the default (mass deficient) extraction

In [ ]:
# set up args for current run
run_name = 'YYYY-MM-DD_run_name'
panel_path = 'C:\\Users\\Customer.ION\\Documents\\panel_files\\my_cool_panel.csv'
extract_prof = True

Everything necessary for and subsequently outputted from this notebook is stored in the automatic directories established in `1_set_up_toffy.ipynb`. More information on the uses and locations of the directories in toffy can be found in the [README](https://github.com/angelolab/toffy#directory-structure).

In [ ]:
# read in panel information
panel = load_panel(panel_path)

# path to the directory containing the FOV bin files
base_dir = os.path.join('D:\\Data', run_name) 

# path to directory containing extracted files
extraction_dir = os.path.join('D:\\Extracted_Images', run_name)    
if not os.path.exists(extraction_dir):
    os.makedirs(extraction_dir)

if extract_prof:
    # path to directory containing mass-proficient extracted files (for long-term storage)
    extraction_prof_dir = os.path.join('D:\\Extracted_Images', run_name + '_proficient')
    if not os.path.exists(extraction_prof_dir):
        os.makedirs(extraction_prof_dir)

## Extract FOV bin files
The function below will skip any previously extracted FOVs and print "Extraction completed!" when done. If you receive a warning message stating "no viable bin files were found", then all of the necessary FOVs have already been extracted.

In [ ]:
# define the specific channels you want to extract as intensity images, leave list empty if there are none
# NOTE: these must match names found in panel_path
intensity_channels = ["Au", "chan_39"]

In [ ]:
# base deficient extraction
extract_missing_fovs(
    base_dir,
    extraction_dir,
    panel,
    extract_intensities=intensity_channels,
)

# mass proficient extraction (for long-term storage)
if extract_prof:
    extract_missing_fovs(
        base_dir,
        extraction_prof_dir,
        modify_panel_ranges(panel, start_offset=0.3, stop_offset=0.3),
        extract_intensities=intensity_channels,
    )

## Check for any incomplete FOVs 
If the instrument is shut off part way through a run, this can result in output FOVs which are generated, but missing counts in parts of the images. The cell below will check for any incompete FOVs and warn you if any exist.

In [ ]:
incomplete_fov_check(base_dir, extraction_dir)